In [1]:
import pandas as pd
import re

In [2]:
file_name = 'phase_turbo.csv'

In [3]:
df = pd.read_csv(file_name)

In [4]:
df


,PMID,Answers
0,30738380,Q: Does the article explicitly mention the cli...
1,31264510,Q: Does the article explicitly mention the cli...
2,25822479,Q: Does the article explicitly mention the cli...
3,26016467,Q: Does the article explicitly mention the cli...
4,17628435,Q: Does the article explicitly mention the cli...
5,27371157,Q: Does the article explicitly mention the cli...
6,9442338,Q: Does the article explicitly mention the cli...
7,33691517,Q: Does the article explicitly mention the cli...
8,8905533,Q: Does the article explicitly mention the cli...
9,23968123,Q: Does the article explicitly mention the cli...


In [18]:
answers = []
for index, row in df.iterrows():
    curr_answer = str(row['Answers'])
    #print(curr_answer)
    query = re.findall("A:\s[\w]*\s*[\d]*", curr_answer)
    #print(query)
    ans = []
    for ele in query:
        ele = ele[3:].lower()
        ans.append(ele)
    answers.append(ans)
print(answers)

[['no', 'unknown\n', 'yes\n', 'yes\n', '227 ', 'no'], ['yes', 'phase 1', 'yes', 'yes', '95', 'no'], ['no\n\n', 'unknown\n\n', 'yes\n\n', 'yes\n\n', '100\n\n', 'no'], ['no', 'unknown', 'yes', 'yes', '100 ', 'no'], ['no', 'unknown', 'yes', 'yes', '73 ', 'no'], ['yes\n\n', 'unknown\n\n', 'yes\n\n', 'yes\n\n', '60\n\n', 'no'], ['no', 'unknown', 'yes', 'yes', '9 ', 'no'], ['no', 'unknown', 'yes', 'yes', 'the ', 'no'], ['no', 'unknown', 'yes', 'yes', '66 ', 'no'], ['yes', 'the ', 'yes', 'yes', 'n', 'no']]
